**Import the required modules**

In [1]:
import torch
if torch.cuda.is_available():
    device = "cuda"
else:
    print ('[WARNING] CUDA/GPU is not available! Compute-intensive scripts on this notebook will be run on CPU.')
    device =  "cpu"

Clone the kcg-ml repo as it has all the required utils and codes from preprocessing the image datasets to training the models and using them to classify the images.

In [2]:
!git clone https://github.com/kk-digital/kcg-ml
%cd kcg-ml

Cloning into 'kcg-ml'...
remote: Enumerating objects: 3051, done.
remote: Counting objects: 100% (1043/1043), done.
remote: Compressing objects: 100% (412/412), done.
remote: Total 3051 (delta 666), reused 954 (delta 626), pack-reused 2008
Receiving objects: 100% (3051/3051), 26.61 MiB | 30.45 MiB/s, done.
Resolving deltas: 100% (1813/1813), done.
/content/kcg-ml


In [ ]:
#@title Download Dataset
%%capture
# Mega CMD Requirements
!apt install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https
!apt --fix-broken install

# Mega CMD Download and Installation
!wget https://mega.nz/linux/MEGAsync/xUbuntu_18.04/amd64/megacmd-xUbuntu_18.04_amd64.deb
!sudo dpkg -i megacmd-xUbuntu_18.04_amd64.deb

import os
import contextlib
from subprocess import Popen, PIPE, STDOUT

# Download URL on Mega
tagged_dataset_url = 'https://mega.nz/file/8RhFRISJ#vlQhjBp5hrNtQzFnRVQtD_ilHfIyLOSrlwVXEb3t1UM'
other_dataset_url  = 'https://mega.nz/file/5FhGkRjD#yFihfhr1RMHfPTffhPB4tQtJsnn_HBYFOSfqdPOrp78'
# Destination path for download
destination_path = './downloads'
os.makedirs(destination_path, exist_ok=True)

# Function for printing the download progress
def print_progress(proc, stream='stdout'):
  newlines = ['\n', '\r\n', '\r']
  stream = getattr(proc, stream)
  with contextlib.closing(stream):
      while True:
          out = []
          last = stream.read(1)
          # Don't loop forever
          if last == '' and proc.poll() is not None:
              break
          while last not in newlines:
              # Don't loop forever
              if last == '' and proc.poll() is not None:
                  break
              out.append(last)
              last = stream.read(1)
          out = ''.join(out)
          yield out

# Download dataset
for data_url in [tagged_dataset_url, other_dataset_url]:
  cmd = ["mega-get", data_url, destination_path]
  proc = Popen(cmd,stdout=PIPE, stderr=STDOUT, universal_newlines=True)
  for line in print_progress(proc):
    print(line)


**Extract Downloaded ZIP-Archived Data**

In [ ]:
from zipfile import ZipFile

# Specify location of downloaded data (zip file)
downloaded_data_zip = "./downloads/pixel-art-tagged-v3.zip"
# Location to extract the zip file to
dataset_path = f'./datasets/{os.path.splitext(os.path.split(downloaded_data_zip)[-1])[0]}'

with ZipFile(downloaded_data_zip) as zip_object:
    zip_object.extractall(dataset_path)

**Loading the Dataset from Drive**

In [3]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/pixel-art-tagged-v3.zip /content/kcg-ml/datasets/
#!unzip /content/pixel-art-tagged-v3.zip

Mounted at /content/drive


In [4]:
%%capture
%pip install ascii_graph open_clip_torch patool fire

**Import ClipModel to generate the CLIP vectors of the images**

In [5]:
from examples.ClipTools import ClipModel

In [6]:
dataset_path = '/content/kcg-ml/datasets/pixel-art-tagged-v3.zip'
output_folder = './output'
tagged_dataset = True
clip_model = 'ViT-L-14'
pretrained = 'laion2b_s32b_b82k'
batch_size = 32
num_threads = 4
device = None

In [7]:
clip_model_instance = ClipModel(clip_model=clip_model, pretrained=pretrained)
clip_model_instance.download_model(clip_model, pretrained)

[INFO] Model downloaded succesfully


Generate CLIP vectors

In [8]:
clip_model_instance.encode_data_directory(dataset_path,output_dir='./datasets')

[INFO] ZIP 1 OF 1, TIME : 29.02 SECS, 297 IMAGES


**Load the CLIP vectors**

In [9]:
import json
# load the dataset and split the dataset into train and test
with open("./datasets/clip-scores.json", "r") as f:
  data = json.load(f)

In [11]:
# get the total number of records
total = len(data)
total

297

In [14]:
# generate the random indices
import numpy as np
indices = np.random.permutation(total)
indices.shape

(297,)

**SPLIT the CLIP vectors into train and test sets (50% each)**

In [15]:
training_idx, test_idx = indices[:int(total*0.5)], indices[int(total*0.5):]

In [16]:
len(training_idx)+len(test_idx)

297

In [17]:
train_data = [data[i] for i in training_idx]
test_data = [data[i] for i in test_idx]